In [40]:
   
    
    from astropy.io import fits
    import numpy as np
    import scipy
    from scipy import ndimage
    import skimage
    from skimage import draw
    import pandas as pd
    import time

    # Loading 4D image which is our given data

    im = 'G327_cvel_06_12M_spw0_line_SiO_5-4'

    data, hd = fits.getdata(im+'.fits', header=True)

    # Slicing and reduing dimensions

    data = data[0,:,:,:]

    data_new = data[:,200:500,600:900]

    # Thresholding values of array

    data_new[data_new < 0.0235] = 0

    #data_new = np.pad(data_new, pad_width = [(0,),(83,),(69,)], mode = 'constant')

    dta = np.zeros_like(data_new)

    # creating an ellipsoid mask

    a,b,c = 14,14,40


    mask = skimage.draw.ellipsoid(a,b,c)

    mask2 = np.pad(mask, pad_width = [((100 -(a-1)),),(100-(b-1),),((100-(c-1)),)] , mode = 'constant')

    # Position of the mask center

    x = 150 - mask2.shape[0]//2       # channel number
    y = 160 - mask2.shape[1]//2       # Dec  Y
    z = 150 - mask2.shape[2]//2       # R.A. X

    # shifting the center to the one end

    shift1 = scipy.ndimage.shift(mask2.astype(int), np.array([0,0,-c]))

    # Selecting anlges to rotate the ellipsoid mask

    angle1 = 77
    angle2 = 342

    #Rotation

    mask3 = scipy.ndimage.interpolation.rotate(shift1,angle1 , axes = (0,2), reshape = False)

    mask4 = scipy.ndimage.interpolation.rotate(mask3, angle2 , axes = (0,1), reshape = False)

    # Inserting masked array inside another empty array

    dta[x:x+mask4.shape[0], y:y+mask4.shape[1], z:z+mask4.shape[2]] = mask4

    # Multiplying the empty array with mask with original data

    data_new2 = dta*data_new

    # Expanding dimension

    data_new3 = np.expand_dims(data_new2, axis = 0)

    #data_newx = np.expand_dims(data_new, axis = 0)

    Intensity  =  np.sum(data_new2)

    print(" Total Intensity of the Outflow lobe is :  ", Intensity)


    # Saving the resulting data

    fits.writeto('Region1outflow1.fits',data_new3, hd , overwrite = True)


 Total Intensity of the Outflow lobe is :   629.7919


In [ ]:
    #import pakages

    from astropy.io import fits
    import numpy as np
    import scipy
    from scipy import ndimage
    import skimage
    from skimage import draw
    import pandas as pd
    import time
    start = time.time()

    # Loading the 3D image which is the data we want to analyse

    im = 'sio'

    data, hd = fits.getdata(im+'.fits', header=True)

    # Slicing the data 

    data = data[0,:,:,:]

    data_new = data[:,200:500,600:900]

    dta = np.zeros_like(data_new)


    # Use ellipsoid as mask

    mask = skimage.draw.ellipsoid(12,12,50)

    mask2 = np.pad(mask, pad_width = 95 , mode = 'constant')

    array = mask2.astype(int)

    # Position on the data where we place the center of  ellipsoid 

    x = 149 - mask2.shape[0]//2       # channel number
    y = 160 - mask2.shape[1]//2       # Dec  Y
    z = 152 - mask2.shape[2]//2       # R.A. X


    # Shifting the ellipsoid center to one end

    shift1 = scipy.ndimage.shift(array, np.array([0,0,-50]))


    Data = []

    # Rotating ellipsoid(mask) through all possible anlges and calculate sum of intesity at each location 

    for angle1 in range(0,180,5):
            mask3 = scipy.ndimage.interpolation.rotate(shift1,angle1 , axes = (0,2), reshape = False)
            for angle2 in range(0,360,5):
                    mask4 = scipy.ndimage.interpolation.rotate(mask3,angle2 , axes = (0,1), reshape = False)
                    dta[x:x+mask4.shape[0], y:y+mask4.shape[1], z:z+mask4.shape[2]] = mask4
                    data_new2 = dta*data_new
                    I_integrated =  np.sum(data_new2)
                    print (angle1, angle2,  I_integrated ,   np.sum(dta))
                    Data.append([round(angle1,2),round(angle2,2), round(I_integrated,2)])

    # Creating datafarame in Pandas

    df = pd.DataFrame(columns = ['angle1','angle2','Flux(Jansky)'], data = Data)
    print (df)

    df.to_csv('ellipsoid8.csv', sep = '\t')


    data_new3 = np.expand_dims(data_new2, axis = 0)

    #saving the file

    fits.writeto('overlap8.fits', data_new3,hd, overwrite = True)

    end = time.time()
    print(end-start)

In [ ]:
    ## plotting the data obtained in the form of .csv file  as heatmap

    import pandas as pd 
    import matplotlib.pyplot as plt
    import seaborn as sns
    import mpldatacursor
    from mpldatacursor import datacursor
    import matplotlib.pyplot as plt
    df = pd.read_csv('ellipsoid8.csv', sep = '\t')
    del df['Unnamed: 0']
    result = df.pivot(index = 'angle1', columns = 'angle2', values = 'Flux(Jansky)')
    sns.heatmap(result)
    plt.savefig("heatmap3.png")
